In [3]:
import pandas as pd

# Cargar el DataFrame (asegúrate de incluir las columnas necesarias)
df = pd.read_csv("df_final.csv")  # Cambia a tu archivo real

# Filtrar artistas consistentes y calcular minutos reproducidos por año
df['date'] = pd.to_datetime(df['ts'])
df['year'] = df['date'].dt.year
df['min_played'] = df['ms_played'] / 60000

# Agrupar por artista y año
df_grouped = df.groupby(['master_metadata_album_artist_name', 'year'])['min_played'].sum().reset_index()

# Filtrar al top 15 más consistentes
consistent_artists = (
    df_grouped.groupby('master_metadata_album_artist_name')['year']
    .nunique()
    .reset_index()
    .query('year >= 7')  # Artistas consistentes al menos 7 años
    .sort_values('year', ascending=False)
    .head(15)['master_metadata_album_artist_name']
)

# Filtrar el DataFrame solo con los artistas consistentes
df_filtered = df_grouped[df_grouped['master_metadata_album_artist_name'].isin(consistent_artists)]


C:\Users\SIMON\AppData\Local\Temp\ipykernel_2684\3427152367.py:4: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("df_final.csv")  # Cambia a tu archivo real


In [ ]:
import plotly.express as px
import nbformat

# Crear el gráfico animado
fig = px.bar(
    df_filtered,
    x='min_played',
    y='master_metadata_album_artist_name',
    color='master_metadata_album_artist_name',
    animation_frame='year',
    orientation='h',
    title="Evolución del Top 15 Artistas Más Consistentes (2017-2024)",
    labels={'min_played': 'Minutos Reproducidos', 'artist': 'Artista', 'year': 'Año'},
    range_x=[0, df_filtered['min_played'].max() + 5000],  # Ajusta el rango
)

# Personalización adicional
fig.update_layout(
    title={'x': 0.5, 'xanchor': 'center'},
    xaxis_title="Minutos Reproducidos",
    yaxis_title=None,
    legend_title=None,
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
)

# Mostrar el gráfico
fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
import plotly.io as pio

# Exportar como GIF
pio.write_image(fig, "timelapse.gif", format="gif", engine="kaleido")
